# Assignment 2

## First draft
### Youssef Ragab, Ting Huang, Garrett Hastings


**1. Loading in the required libraries:**

In [6]:
import pandas as pd 
import matplotlib.pyplot as plt 
import h2o 
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch 
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator 
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.frame import H2OFrame      
import itertools       
import matplotlib.pyplot as plt        
import xgboost as xgb 
import time 
import seaborn as sns 
import numpy as np
import datetime

Initializations: 

In [7]:
Seed = 2021

nthreads = 4

h2o.init(max_mem_size= '6G', nthreads= nthreads)
h2o.remove_all()
h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,48 mins 49 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_ghast_arvo6t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.948 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Start global timer:

In [8]:
tic = time.time()

Importing data: 

In [38]:
train = pd.read_csv('hmda_train_preprocessed.csv')
test = pd.read_csv('hmda_test_preprocessed.csv')

In [39]:
train.columns

Index(['row_id', 'black', 'asian', 'white', 'amind', 'hipac', 'hispanic',
       'non_hispanic', 'male', 'female', 'agegte62', 'agelt62', 'term_360',
       'conforming', 'debt_to_income_ratio_missing', 'loan_amount_std',
       'loan_to_value_ratio_std', 'no_intro_rate_period_std',
       'intro_rate_period_std', 'property_value_std', 'income_std',
       'debt_to_income_ratio_std', 'high_priced'],
      dtype='object')

In [40]:
test.columns

Index(['row_id', 'black', 'asian', 'white', 'amind', 'hipac', 'hispanic',
       'non_hispanic', 'male', 'female', 'agegte62', 'agelt62', 'term_360',
       'conforming', 'debt_to_income_ratio_missing', 'loan_amount_std',
       'loan_to_value_ratio_std', 'no_intro_rate_period_std',
       'intro_rate_period_std', 'property_value_std', 'income_std',
       'debt_to_income_ratio_std'],
      dtype='object')

Now that we have the datasets loaded, we can proceed and remove the columns that we don't need from the training set. Based on the assignment prompt, we are going to keep the following variables:
* high_priced
* conforming 
* debt_to_income_ratio_std
* debt_to_income_ratio_missing
* income_std 
* loan_amount_std
* intro_rate_period_std
* loan_to_value_ratio_std
* no_intro_rate_period_std
* property_value_std
* term_360

In [41]:
x_name = ['conforming', 'debt_to_income_ratio_std', 'debt_to_income_ratio_missing', 'income_std', 'loan_amount_std', 'intro_rate_period_std', 'loan_to_value_ratio_std', 'no_intro_rate_period_std', 'property_value_std', 'term_360']
y_name = 'high_priced'



In [42]:
#Removing this so that 'row_id' is in the dataframe later.

#train = train[x_name + [y_name]]

Now that we have only kept the columns that we need from the training set, we can do the same for the testing or validation set. 

In [43]:
#Removing this so that 'row_id' is in the dataframe later.

#test = test[x_name]
#test.columns

### Data exploration:

**Histograms:**

In [44]:
#_ = train.hist(bins = 50, figsize= (15,15))

**Correlation:**

In [45]:
#_ = sns.heatmap(train.corr(),
                #xticklabels= train.columns.values,
                #yticklabels= train.columns.values)

Since the testing set does not have the y column, there is no need to add it, and we are left with the same variables as the training set. 

We can move ahead to training the model. 

### Fitting interpretable models: 

**Splitting data into training and validation subsets:**

In [46]:
np.random.seed(Seed)

split_ratio = 0.7

split = np.random.rand(len(train)) < split_ratio

training = train[split]
validation = train[~split]

print('Train data rows = %d, columns = %d' % (training.shape[0], training.shape[1]))
print('Train data rows = %d, columns = %d' % (validation.shape[0], validation.shape[1])) 


Train data rows = 112085, columns = 23
Train data rows = 48253, columns = 23


### Elastic Net

**Defining wrapper function for hyperparameter grid search:**

In [47]:
def glm_grid(x_name, y_name, htrain, hvalid, seed):

    # Creating alpha options to set L2:
    alpha_options = [0.1, 0.25, 0.5, 0.99]
    
    # Defining serch criteria
    hyperparameters = {'alpha': alpha_options}

    # Initializing grid 
    grid = H2OGridSearch(
        H2OGeneralizedLinearEstimator(family = 'binomial',
                                      lambda_search = True,
                                      seed = seed),
        hyper_params = hyperparameters)

    # Executing training with grid search
    grid.train(y = y_name,
               x = x_name,
               training_frame = htrain, 
               validation_frame = hvalid,
               seed = seed)

    # Choosing the best model from the grid seach 
    best_model = grid.get_grid()[0]
    del grid 

    return best_model


**Fitting elastic net:**

In [48]:
# Start timer 
glm_tic = time.time()

# Converting data into H2O Frame 
htrain = h2o.H2OFrame(training)
hvalid = h2o.H2OFrame(validation)

# Training with grid search 
best_glm = glm_grid(x_name, y_name, htrain, hvalid, Seed)

# End timer 
glm_toc = time.time() - glm_tic
print('Elastic net GLM training completed in %.2f s.' % glm_toc)

Elastic net GLM training completed in 45.11 s.


**Best AUC assessment:**

In [49]:
print('Validation AUC: %.4f.' % best_glm.auc(valid = True))

Validation AUC: 0.7563.


**Writing submission files:**

In [52]:
best_glm_submit = best_glm.predict(h2o.H2OFrame(test)).as_data_frame()
best_glm_submit.drop(['predict', 'p0'], axis = 1, inplace = True)
best_glm_submit.columns = ['phat']
best_glm_submit.to_csv('best_glm_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                        index=False)

In [53]:
best_glm_submit.head()

,phat
0,0.143552
1,0.081619
2,0.128237
3,0.005958
4,0.131587


# NEW CODE

#### Utility function for selecting percentiles

In [54]:
best_glm_submit = pd.concat([validation.reset_index(drop=True), best_glm_submit], axis=1)
best_glm_submit.head()

,row_id,black,asian,white,amind,hipac,hispanic,non_hispanic,male,female,...,debt_to_income_ratio_missing,loan_amount_std,loan_to_value_ratio_std,no_intro_rate_period_std,intro_rate_period_std,property_value_std,income_std,debt_to_income_ratio_std,high_priced,phat
0,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0,-0.118642,0.268727,0.244394,-0.215304,-0.227585,-0.018133,-0.425131,0,0.143552
1,4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,...,0,-0.602338,0.552520,0.244394,-0.215304,-0.628437,-0.038228,0.763191,0,0.081619
2,7,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0,-0.734255,0.552520,0.244394,-0.215304,-0.720941,-0.039614,0.488963,0,0.128237
3,9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0,1.332446,-0.261719,0.244394,-0.215304,1.190815,0.075764,-1.156406,0,0.005958
4,12,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0,-0.514393,0.333922,0.244394,-0.215304,-0.535932,-0.024369,-0.425131,1,0.131587


In [55]:
def get_percentile_dict(yhat_name, frame, id_):

    """ Returns the percentiles of a column, yhat_name, as the indices based on
        another column id_.
        :param yhat_name: Name of column in frame in which to find percentiles.
        :param frame: Pandas frame.
        :param id_: Validation Pandas frame containing yhat and id_.
        :return: Dictionary of percentile values and index column values.
    """

    # create a copy of frame and sort it by yhat
    sort_df = frame.copy(deep=True)
    sort_df.sort_values(yhat_name, inplace=True)
    sort_df.reset_index(inplace=True)

    # find top and bottom percentiles
    percentiles_dict = {0: sort_df.loc[0, id_], 99: sort_df.loc[sort_df.shape[0] - 1, id_]}

    # find 10th-90th percentiles
    inc = sort_df.shape[0] // 10
    for i in range(1, 10):
        percentiles_dict[i * 10] = sort_df.loc[i * inc, id_]

    return percentiles_dict

#### Select percentiles from predictions

In [58]:
best_glm_percentiles = get_percentile_dict('phat', best_glm_submit, 'row_id')
for key in sorted(best_glm_percentiles.keys()):
    print(str(key) + ': ' + str(best_glm_percentiles[key]))

0: 15340
10: 65757
20: 42634
30: 49003
40: 20879
50: 80111
60: 96502
70: 112762
80: 128906
90: 144457
99: 160335


#### Collect global feature importance info

In [59]:
# use coefficients as relative global feature importance
global_feat_imp = pd.DataFrame.from_dict(best_glm.coef(), columns=['GLM Importance'], orient='index')
global_feat_imp.drop('Intercept', inplace=True)
global_feat_imp['GLM Importance'] = np.abs(global_feat_imp['GLM Importance'])

#### Collect local feature importance info

In [60]:
# use (coefficient * value) as local feature importance

# init frame to store local contribs
local_fi_dict = {10: pd.DataFrame(columns = ['GLM Contribution'], index=x_name),
                 50: pd.DataFrame(columns = ['GLM Contribution'], index=x_name),
                 90: pd.DataFrame(columns = ['GLM Contribution'], index=x_name)}

# get (coefficient * value) at three percentiles of phat
for name in x_name:
    for percentile in [10, 50, 90]:
    
        # local contributions = beta_j * x_i,j
        local_fi_dict[percentile].loc[name, 'GLM Contribution'] =\
            best_glm.coef()[name] *\
            validation[validation['row_id'] == int(best_glm_percentiles[percentile])][name].values[0]

In [68]:
#local_fi_dict[10]
local_fi_dict[50]
#local_fi_dict[90]

,GLM Contribution
conforming,0
debt_to_income_ratio_std,0.131437
debt_to_income_ratio_missing,0
income_std,0
loan_amount_std,0
intro_rate_period_std,0
loan_to_value_ratio_std,-0.0500465
no_intro_rate_period_std,-0
property_value_std,-2.20074
term_360,0.222036


### Monotonic XGBoost
**Define utility function for random grid search**

In [15]:
def xgb_grid(dtrain, dvalid, mono_constraints=None, gs_params=None, n_models=None,
             ntree=None, early_stopping_rounds=None, verbose=False, seed=None):
    

    # cartesian product of gs_params
    keys, values = zip(*gs_params.items())
    experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # preserve exact reproducibility for this function
    np.random.seed(Seed) 
    
    # select randomly from cartesian product space
    selected_experiments = np.random.choice(len(experiments), n_models)

    # set global params for objective,  etc.
    params = {'booster': 'gbtree',
              'eval_metric': 'auc',
              'nthread': nthreads,
              'objective': 'binary:logistic',
              'seed': Seed}

    # init grid search loop
    best_candidate = None
    best_score = 0

    # grid search loop
    for i, exp in enumerate(selected_experiments):

        params.update(experiments[exp])  # override global params with current grid run params

        print('Grid search run %d/%d:' % (int(i + 1), int(n_models)))
        print('Training with parameters:', params)

        # train on current params
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        
        if mono_constraints is not None:
            params['monotone_constraints'] = mono_constraints
        
        candidate = xgb.train(params,
                              dtrain,
                              ntree,
                              early_stopping_rounds=early_stopping_rounds,
                              evals=watchlist,
                              verbose_eval=verbose)    

        # determine if current model is better than previous best
        if candidate.best_score > best_score:
            best_candidate = candidate
            best_score = candidate.best_score
            print('Grid search new best score discovered at iteration %d/%d: %.4f.' %
                             (int(i + 1), int(n_models), candidate.best_score))

        print('---------- ----------')
            
    return best_candidate

**Fit monotonic XGBoost with random grid search**

In [ ]:
# dictionary of hyperparameter value lists for grid search
gs_params = {'colsample_bytree': [0.3, 0.5, 0.7, 0.9],
             'colsample_bylevel': [0.3, 0.5, 0.7, 0.9],
             'eta': [0.005, 0.05, 0.5],
             'max_depth': [3, 5, 7],
             'reg_alpha': [0.0005, 0.005, 0.05],
             'reg_lambda': [0.0005, 0.005, 0.05],
             'subsample': [0.3, 0.5, 0.7, 0.9],
             'min_child_weight': [1, 5, 10], 
             'gamma': [0.0, 0.1, 0.2 , 0.3, 0.4]}

# define monotonicity constraints
mono_constraints = tuple([int(i) for i in np.sign(train[x_name + [y_name]].corr()[y_name].values[:-1])])

# start local timer
mxgb_tic = time.time()

# Convert data to SVMLight format
dtrain = xgb.DMatrix(training[x_name], training[y_name])
dvalid = xgb.DMatrix(validation[x_name], validation[y_name])

# Monotonic XGBoost grid search
best_mxgb = xgb_grid(dtrain, dvalid, gs_params=gs_params, n_models=50, ntree=1000, early_stopping_rounds=100, 
                     mono_constraints=mono_constraints, seed=Seed)

# end local timer
mxgb_toc = time.time() - mxgb_tic
print('Monotonic GBM training completed in %.2f s.' % (mxgb_toc))

Grid search run 1/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'auc', 'nthread': 4, 'objective': 'binary:logistic', 'seed': 2021, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.5, 'eta': 0.005, 'max_depth': 7, 'reg_alpha': 0.005, 'reg_lambda': 0.005, 'subsample': 0.5, 'min_child_weight': 5, 'gamma': 0.1}
Grid search new best score discovered at iteration 1/50: 0.7708.
---------- ----------
Grid search run 2/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'auc', 'nthread': 4, 'objective': 'binary:logistic', 'seed': 2021, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.3, 'eta': 0.05, 'max_depth': 5, 'reg_alpha': 0.05, 'reg_lambda': 0.05, 'subsample': 0.5, 'min_child_weight': 10, 'gamma': 0.4, 'monotone_constraints': (1, 1, 1, -1, -1, -1, 1, 1, -1, 1)}
Grid search new best score discovered at iteration 2/50: 0.7875.
---------- ----------
Grid search run 3/50:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'auc', 'nthread': 4, 'obj

**Basic AUC assessment**

In [ ]:
print('Validation AUC: %.4f.' % best_mxgb.best_score)

**Write submission file**

In [ ]:
dtest = xgb.DMatrix(test[x_name])
best_mxgb_submit = pd.DataFrame(best_mxgb.predict(dtest, ntree_limit=best_mxgb.best_ntree_limit), columns=['phat'])
best_mxgb_submit.to_csv('ph_best_mxgb_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                        index=False)

In [ ]:
best_mxgb_submit

### Random forest 

In [ ]:
def feature_combiner(training_frame, test_frame, nums):
    
    """ Combines numeric features using simple arithmatic operations.
    
    :param training_frame: Training frame from which to generate features and onto which generated 
                           feeatures will be cbound.
    :param test_frame: Test frame from which to generate features and onto which generated 
                       feeatures will be cbound.
    :param nums: List of original numeric features from which to generate combined features.
    
    """

    total = len(nums)
    
    # convert to pandas
    train_df = training_frame.as_data_frame()
    test_df = test_frame.as_data_frame()
    
    for i, col_i in enumerate(nums):
        
        print('Combining: ' + col_i + ' (' + str(i+1) + '/' + str(total) + ') ...')        
        
        for j, col_j in enumerate(nums):
            
            # don't repeat (i*j = j*i)
            if i < j:
                
                # convert to pandas
                col_i_train_df = train_df[col_i]
                col_j_train_df = train_df[col_j]
                col_i_test_df = test_df[col_i]
                col_j_test_df = test_df[col_j] 

                # multiply, convert back to h2o
                train_df[str(col_i + '|' + col_j)] = col_i_train_df.values*col_j_train_df.values
                test_df[str(col_i + '|' + col_j)] = col_i_test_df.values*col_j_test_df.values
                
    print('Done.')
    
    # convert back to h2o
    
    print('Converting to H2OFrame ...')
    
    training_frame = h2o.H2OFrame(train_df)
    training_frame.columns = list(train_df)
    test_frame = h2o.H2OFrame(test_df)
    test_frame.columns = list(test_df)
    
    print('Done.')
    print()
    
    # conserve memory 
    del train_df
    del test_df 
    
    return training_frame, test_frame

In [ ]:
H2OFrame(training[x_name])

In [ ]:
rf_model1 = H2ORandomForestEstimator(
    ntrees=10000,                    
    max_depth=10, 
    col_sample_rate_per_tree=0.1,
    sample_rate=0.8,
    stopping_rounds=50,
    score_each_iteration=True,
    nfolds=3,
    keep_cross_validation_predictions=True,
    seed=12345)           

# start timer 
rf_tic = time.time()

# train rf model
rf_model1.train(
    y = 'high_priced',
    training_frame=H2OFrame(training),
    validation_frame=H2OFrame(validation))

# print model information
rf_toc = time.time() - rf_tic
print('Random Forest training completed in %.2f s.' % rf_toc)
print(rf_model1)


rf_preds1_test = rf_model1.predict(H2OFrame(test))
gen_submission(rf_preds1_test) # 0.14574 public leaderboard

In [25]:
best_rf_submit = rf_model1.predict(H2OFrame(test)).as_data_frame()
best_rf_submit.to_csv('best_rf_model_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                        index=False)


In [26]:
best_rf_submit

,predict
0,0.100704
1,0.098442
2,0.119991
3,0.065078
4,0.117926
...,...
19826,0.124180
19827,0.105582
19828,0.127146
19829,0.056343
